# NOTEBOOK DESCRIPTION
The goal of this notebook is to create final ensembles of localized models. Keep in mind that you need a lot of memory for this calculations (we used AWS instance ml.c5.18xlarge).

# LIBS

In [107]:
import pandas as pd
import numpy as np
import os
import pickle
import warnings
import random

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import *
from tensorflow.keras.regularizers import *

cwd = os.path.dirname(os.getcwd())

# DEFAULT
tensorflow_helper_func_path =    '"{}/modules/tensorflow_helper_func.py"'.format(cwd)
%run $tensorflow_helper_func_path

# FUNCTIONS

In [108]:
def get_clust_preds(n_clusters, set_type="test"):
    y_pred = []
    for clust_n in range(n_clusters):
        file_path = folder_mod_local + "y_{}_pred/y_{}_pred, n_clusters={}, clust={}.p".format(set_type, set_type, n_clusters, clust_n)
        y_pred_clust = pd.read_pickle(file_path)
        y_pred.append(y_pred_clust)

    y_pred = pd.concat(y_pred).clip(0)
    return y_pred

def get_errors(y_true, y_pred, squared=False):
    ts_id = y_true.loc[:, "ts_id"]
    errors = y_true.loc[:, y_true.columns.difference(["ts_id"])] - y_pred.loc[:, y_pred.columns.difference(["ts_id"])]
    
    if squared: errors = errors ** 2
    
    errors = pd.concat([ts_id, errors], axis=1)
    return errors

def get_mae(errors):
    mae_df = errors.abs().groupby("ts_id").mean()
    mae_per_ts = mae_df.mean(axis=1)
    return mae_per_ts

def get_mape(errors, y_true):
    ts_id = y_true.loc[:, "ts_id"]

    pe = 100 * errors.loc[:, errors.columns.difference(["ts_id"])].abs() / y_true.loc[:, y_true.columns.difference(["ts_id"])]
    pe = pd.concat([ts_id, pe.mean(axis=1)], axis=1)

    mape_per_ts = pe.groupby("ts_id").mean().iloc[:, 0]
    return mape_per_ts

def get_nmae(errors, y_true):
    mae_df = errors.abs().groupby("ts_id").mean()
    mae_per_ts = mae_df.mean(axis=1)
    mean_per_ts = y_true.groupby("ts_id").mean().mean(axis=1)
    return mae_per_ts / mean_per_ts

def mean_per_agg(s):
    s_per_agg = pd.Series({"single": s.iloc[:250].mean(), 
                           "sTS":    s.iloc[250: 500].mean(),
                           "mTS":    s.iloc[500: 750].mean(),
                           "lTS":    s.iloc[750:].mean(),
                           "All":    s.mean()
                          })
    return s_per_agg.round(4)

# 0. INPUT DATA

In [109]:
folder_gen = cwd + "/generated_data/"
folder_mod_global = cwd + "/models/global N-BEATS-exog/"
folder_mod_local = cwd + "/models/localized N-BEATS-exog/"

# CLUSTERING RESULTS
clust_all = pd.read_csv(folder_gen + "cluster_data.csv", index_col="ts_id")
clust_all.columns = clust_all.columns.astype(int)

# TRUE TARGET VALUES
y_val = pd.read_pickle(folder_gen + "y_val.p")
y_test = pd.read_pickle(folder_gen + "y_test.p")

# NAIVE PREDICTIONS
y_val_pred_naive = pd.read_pickle(folder_gen + "y_val_pred_naive.p")
y_test_pred_naive = pd.read_pickle(folder_gen + "y_test_pred_naive.p")

# INITIAL GLOBAL MODEL PREDICTIONS
y_val_pred_baseline = pd.read_pickle(folder_mod_global + "y_val_pred.p")
y_test_pred_baseline = pd.read_pickle(folder_mod_global + "y_test_pred.p")

periods = 48
y_cols = ["H_{}".format(i) for i in range(1, periods+1)]

# 1. LOAD & EVALUATE PREDICTIONS

In [110]:
# EVALUATE NAIVE MODEL
val_errors_naive = get_errors(y_val, y_val_pred_naive)
val_mae_naive = get_mae(val_errors_naive)

test_errors_naive = get_errors(y_test, y_test_pred_naive)
test_mae_naive = get_mae(test_errors_naive)
test_mae_naive_per_horizon = test_errors_naive.abs().groupby("ts_id").mean()

In [ ]:
max_n_clusters = 20

y_val_pred_clust_dict, y_test_pred_clust_dict = {}, {}

val_mae, test_mae = [], []
val_nmae, test_nmae = [], []
val_mape, test_mape = [], []
val_mase, test_mase = [], []

# CREATE PREDICTION DICT & EVALUATION PER TS
for n_clusters in range(1, max_n_clusters+1):
    print(n_clusters)
    
    # LOAD PREDICTIONS
    if n_clusters == 1:  # load predictions from initial global model
        y_val_pred_clust = y_val_pred_baseline
        y_test_pred_clust = y_test_pred_baseline
        
        y_val_pred_clust_dict[n_clusters] = y_val_pred_baseline
        y_test_pred_clust_dict[n_clusters] = y_test_pred_baseline
        
    else:  # load predictions from localized models
        y_val_pred_clust = get_clust_preds(n_clusters, set_type="val")
        # re-order rows to match y_val
        y_val_pred_clust = (y_val.loc[:, ["ts_id"]].reset_index()
                            .merge(y_val_pred_clust.reset_index(), on=["timestamp", "ts_id"])
                            .set_index("timestamp"))
        
        y_test_pred_clust = get_clust_preds(n_clusters, set_type="test")
        # re-order rows to match y_test
        y_test_pred_clust = (y_test.loc[:, ["ts_id"]].reset_index()
                            .merge(y_test_pred_clust.reset_index(), on=["timestamp", "ts_id"])
                            .set_index("timestamp"))
            
        y_val_pred_clust_dict[n_clusters] = y_val_pred_clust
        y_test_pred_clust_dict[n_clusters] = y_test_pred_clust

        
    # EVALUATION - val
    val_errors_clust = get_errors(y_val, y_val_pred_clust)

    ## MAE
    val_mae_clust = get_mae(val_errors_clust)
    val_mae.append(val_mae_clust.rename(n_clusters))

    ## NMAE
    val_nmae_clust = get_nmae(val_errors_clust, y_val)
    val_nmae.append(val_nmae_clust.rename(n_clusters))

    ## MAPE
    val_mape_clust = get_mape(val_errors_clust, y_val)
    val_mape.append(val_mape_clust.rename(n_clusters))

    ## MASE
    val_mase_clust = val_mae_clust / val_mae_naive
    val_mase.append(val_mase_clust.rename(n_clusters))
        
        
    # EVALUATION - test
    test_errors_clust = get_errors(y_test, y_test_pred_clust)

    ## MAE
    test_mae_clust = get_mae(test_errors_clust)
    test_mae.append(test_mae_clust.rename(n_clusters))

    ## NMAE
    test_nmae_clust = get_nmae(test_errors_clust, y_test)
    test_nmae.append(test_nmae_clust.rename(n_clusters))

    ## MAPE
    test_mape_clust = get_mape(test_errors_clust, y_test)
    test_mape.append(test_mape_clust.rename(n_clusters))

    ## MASE
    test_mase_clust = test_mae_clust / test_mae_naive
    test_mase.append(test_mase_clust.rename(n_clusters))
    
val_mae = pd.concat(val_mae, axis=1)
val_nmae = pd.concat(val_nmae, axis=1)
val_mape = pd.concat(val_mape, axis=1)
val_mase = pd.concat(val_mase, axis=1)
    
test_mae = pd.concat(test_mae, axis=1)
test_nmae = pd.concat(test_nmae, axis=1)
test_mape = pd.concat(test_mape, axis=1)
test_mase = pd.concat(test_mase, axis=1)

# 2. PROPOSED LOCALIZATION WITH ENSEMBLING
- create an ensemble from the cluster hierarchy, as proposed in our framework (denoted as ENS)

In [ ]:
ts_res_all = []

for ts_id in y_val.ts_id.unique():
    if ts_id % 100 == 0: print(ts_id)
       
    # create a list of plausible clusters
    # clust_list[0] corresponds to initial global model
    # clust_list[1] corresponds to using k = 2 when clustering (2 subsets of initial set of time series)
    # keep in mind, clust_list is sorted according to performance on VAL set (val_mase) !!!
    clust_list = val_mase.loc[ts_id].sort_values().index

    # baseline MASE (without ensembling) - same as initial global model
    mase_ts_previous = val_mase.loc[ts_id, clust_list[0]]
    optim_n_candidates = 1

    # DETERMINE BEST CANDIDATES ON VAL
    val_mase_ts_list = [mase_ts_previous]

    # try different number of candidates
    # start with 2 models (initial global model and 2 cluster subsets), 
    # next 3 models (initial global model and 2 cluster subsets and 3 cluster subsets) etc.
    for n_candidates in range(2, len(clust_list)+1):
        candidates_list = clust_list[:n_candidates]

        ensemble_list = []
        for n_clusters in candidates_list:
            y_val_pred_ts = y_val_pred_clust_dict[n_clusters]
            y_val_pred_ts = y_val_pred_ts.loc[y_val_pred_ts.ts_id == ts_id, "H_1":]
            ensemble_list.append(y_val_pred_ts)

        y_val_pred_ts_ensemble = pd.concat(ensemble_list).resample("30min").mean().dropna()
        y_val_pred_ts_ensemble = pd.concat([y_val.loc[y_val.ts_id == ts_id, "ts_id"],
                                            y_val_pred_ts_ensemble
                                           ], axis=1)
        
        val_mae_ts = (y_val.loc[y_val.ts_id == ts_id, :].iloc[:, 1:] - y_val_pred_ts_ensemble.iloc[:, 1:]).abs().values.mean()
        val_mase_ts = val_mae_ts / val_mae_naive.loc[ts_id]
        val_mase_ts_list.append(val_mase_ts)

        # if MASE improvement > 0.1 % go further, otherwise exit from for loop
        if (mase_ts_previous - val_mase_ts) > 0.001: 
             mase_ts_previous = val_mase_ts
        else:
            best_candidates_list = candidates_list[:-1]  # save all except the last one
            break

    # EVALUATE ON TEST (after determining best_candidates_list on VAL set)
    ## create ensemble predictions
    y_test_pred_ts_ensemble = []

    for n_clusters in best_candidates_list:
        y_test_pred_ts = y_test_pred_clust_dict[n_clusters]
        y_test_pred_ts = y_test_pred_ts.loc[y_test_pred_ts.ts_id == ts_id, "H_1":]
        y_test_pred_ts_ensemble.append(y_test_pred_ts)

    y_test_pred_ts_ensemble = pd.concat(y_test_pred_ts_ensemble).resample("30min").mean()
    y_test_pred_ts_ensemble = pd.concat([y_test.loc[y_test.ts_id == ts_id, "ts_id"],
                                         y_test_pred_ts_ensemble
                                        ], axis=1)
    
    # EVALUATION OF A FINAL MODEL
    ## final ensemble predictions for each time series are stored in y_test_pred_ts_ensemble
    y_true = y_test.loc[y_test.ts_id == ts_id, :].iloc[:, 1:]
    y_pred = y_test_pred_ts_ensemble.iloc[:, 1:]
    
    ae_ts = (y_true - y_pred).abs()
    ape_ts = 100 * ae_ts / y_true
    test_mape_ts = ape_ts.values.mean()
    
    test_mae_ts = ae_ts.values.mean()
    test_mase_ts = test_mae_ts / test_mae_naive.loc[ts_id]
    
    ts_res = pd.Series({"ts_id": ts_id, 
                        "test_mase": test_mase_ts,
                        "test_mape": test_mape_ts,
                        "test_mae": test_mae_ts, 
                        "best_n_candidates": len(best_candidates_list)})
    ts_res_all.append(ts_res)
    
# SAVE
## test MASE, MAPE & MAE together with number of candidates in ensemble are stored in ts_res_all
ts_res_all = pd.concat(ts_res_all, axis=1).T
ts_res_all = ts_res_all.astype({"ts_id": np.int32, "best_n_candidates": np.int32})